In [1]:
import os
import pandas as pd
import json
from flask import Flask, render_template, request, jsonify, session, send_file
from werkzeug.utils import secure_filename
import io
import uuid

In [ ]:

app = Flask(__name__)
app.secret_key = os.urandom(24)
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024 
app.config['UPLOAD_FOLDER'] = os.path.join(os.path.dirname(os.path.abspath("test.csv")), 'uploads')

os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400
    
    file = request.files['file']
    
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400
    
    if file and file.filename.endswith('.csv'):
        try:
            df = pd.read_csv(file)
            
            if 'Sentences' not in df.columns:
                return jsonify({'error': 'CSV must contain a "Sentences" column'}), 400
            
            label_cols = ['E', 'S', 'G', 'N']
            for col in label_cols:
                if col not in df.columns:
                    df[col] = 0  
                else:
                    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
                    df[col] = df[col].apply(lambda x: 1 if x == 1 else 0)
            
            session_id = str(uuid.uuid4())
            
            temp_file_path = os.path.join(app.config['UPLOAD_FOLDER'], f"{session_id}.csv")
            df.to_csv(temp_file_path, index=False)
            
            session['csv_session_id'] = session_id
            
            counts = {col: int(df[col].sum()) for col in label_cols}
            
            return jsonify({
                'success': True,
                'message': 'File uploaded successfully',
                'rows': len(df),
                'counts': counts
            })
            
        except Exception as e:
            return jsonify({'error': f'Error processing CSV: {str(e)}'}), 500
    
    return jsonify({'error': 'Invalid file format. Please upload a CSV file'}), 400

@app.route('/get_sentence', methods=['GET'])
def get_sentence():
    index = request.args.get('index', 0, type=int)
    
    if 'csv_session_id' not in session:
        return jsonify({'error': 'No CSV file loaded'}), 400
    
    try:
        session_id = session['csv_session_id']
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], f"{session_id}.csv")
        df = pd.read_csv(file_path)
        
        if index < 0 or index >= len(df):
            return jsonify({'error': 'Index out of range'}), 400
        
        sentence = df.loc[index, 'Sentences']
        labels = {col: int(df.loc[index, col]) for col in ['E', 'S', 'G', 'N']}
        
        return jsonify({
            'success': True,
            'sentence': sentence,
            'labels': labels,
            'total': len(df),
            'current': index + 1
        })
        
    except Exception as e:
        return jsonify({'error': f'Error retrieving sentence: {str(e)}'}), 500

@app.route('/update_sentence', methods=['POST'])
def update_sentence():
    if 'csv_session_id' not in session:
        return jsonify({'error': 'No CSV file loaded'}), 400
    
    try:
        data = request.json
        index = data.get('index', 0)
        sentence = data.get('sentence', '')
        labels = data.get('labels', {})
        
        session_id = session['csv_session_id']
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], f"{session_id}.csv")
        df = pd.read_csv(file_path)
        
        if index < 0 or index >= len(df):
            return jsonify({'error': 'Index out of range'}), 400
        
        df.loc[index, 'Sentences'] = sentence
        for col, value in labels.items():
            if col in df.columns:
                df.loc[index, col] = 1 if value else 0
        
        df.to_csv(file_path, index=False)
        
        counts = {col: int(df[col].sum()) for col in ['E', 'S', 'G', 'N']}
        
        return jsonify({
            'success': True,
            'message': 'Sentence updated successfully',
            'counts': counts
        })
        
    except Exception as e:
        return jsonify({'error': f'Error updating sentence: {str(e)}'}), 500

@app.route('/export', methods=['GET'])
def export_csv():
    if 'csv_session_id' not in session:
        return jsonify({'error': 'No CSV file loaded'}), 400
    
    try:
        session_id = session['csv_session_id']
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], f"{session_id}.csv")
        
        return send_file(
            file_path,
            mimetype='text/csv',
            as_attachment=True,
            download_name='labeled_data.csv'
        )
        
    except Exception as e:
        return jsonify({'error': f'Error exporting CSV: {str(e)}'}), 500

@app.route('/get_stats', methods=['GET'])
def get_stats():
    if 'csv_session_id' not in session:
        return jsonify({'error': 'No CSV file loaded'}), 400
    
    try:
        session_id = session['csv_session_id']
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], f"{session_id}.csv")
        df = pd.read_csv(file_path)
        
        counts = {col: int(df[col].sum()) for col in ['E', 'S', 'G', 'N']}
        
        return jsonify({
            'success': True,
            'total': len(df),
            'counts': counts
        })
        
    except Exception as e:
        return jsonify({'error': f'Error getting stats: {str(e)}'}), 500



In [3]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.23.104:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

D:\Anaconda\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
